# Noise propagation in HQNSL quantum loss
In practice the circuit preparing the quantum state for which we wish to learn the symmetries of will be noisy, resulting in a state that is
not quite exactly the state we expect. Possible errors include a depolarizing error, unitary gate (e.g. Pauli), and so forth. Measurement of the quantum
loss function propagates the effect of the error classically into the metric value itself. 

A practical symmetry learning scheme must be robust, at least to some threshold, to such errors. It suffices to show that the loss function does not change significantly enough on noisy states than when the state is prepared without an `e` amount of noise. To demonstrate this, we use the generalized GHZ state. In `HQN.ipynb`, we can find some symmetries of the state both with and without noise. In this notebook, we will apply the noiseless symmetries to the noisy state and check that the quantum loss metric is still small. We will also apply the noisy symmetries to the noiseless state and check likewise that the loss is small. If both cases hold, we conclude that the learning scheme is robust to `e`-noisy states.

State preparation is not the only source of noise. The parametrized quantum circuit (PQC) that implements the symmetry operation can itself have errors similar to those in the state preparation circuit. This poses a separate robustness challenge---can the learning scheme produce valid symmetries when the circuit is itself noisy? This question requires adding a noise parameter to the PQC but does not otherwise differ in any sense to normal learning (one must simply check, as they would usually do, that the learning can converge to a negligbly small quantum loss metric.) Consequently, we defer this latter question to `HQN.ipynb`.